In [1]:
# Load the gams extension
%load_ext gams_magic

# Advertising Budget for Weasley's Wizard Wheezes

The Weasley's are buying advertising time on the radio in order to
reach wizards and witches.  There are a set $N$ of adds that they
could possibly purchase.  Each minute of advertising of type $j \in
N$ costs $c_j$.  If they purchase $x_j$ minutes of advertising of an
add of type $j$, then they will ensure that $\alpha_j \sqrt{x_j}$
witches and $\beta_j \sqrt{x_j}$ wizards hear their spots.

They would like to find a least cost advertising strategy so that
their ads will reach $K_1$ witches and $K_2$ wizards.

For both problems, please use the following gams code to create the instance.

In [2]:
%%gams
sets N /ad1*ad20/;
alias(I,N) ;

option seed = 1253;

parameters 
  c(I) Cost
  alpha(I) Witches proportionality constant
  beta(I) Wizards proportionality constant;

scalars K1, K2, totalAdTime;

c(I) = normal(100,5);

alpha(I) = uniform(7,13) ;
beta(I) = 13-alpha(I) + 7 + 5$(uniform(0,1) < 0.3);

K1 = 5000;
K2 = 8000;

### Write a GAMS nlp model that will determine the number of minutes of each advertising spot to purchase. 

Ensure your code works with the nlp solvers knitro, conopt and ipopt

In [3]:
%%gams
positive variable x(I);
variable objnlp;

equation eqnlpobj;
eqnlpobj.. objnlp =e= sum(I, x(I)*c(I) ); 

equation eqk1;
eqk1.. K1 =l= sum(I, alpha(I)*sqrt(x(I)) ); 

equation eqk2;
eqk2.. K2 =l= sum(I, beta(I)*sqrt(x(I)) );

model advert /eqnlpobj,eqk1,eqk2/;


parameter result(*,*);

option nlp=knitro;
solve advert using NLP min objnlp;
result("knitro","ObjVal") = objnlp.L;

option nlp=conopt;
solve advert using NLP min objnlp;
result("conopt","ObjVal") = objnlp.L;

option nlp=ipopt;
solve advert using NLP min objnlp;
result("ipopt","ObjVal") = objnlp.L;

display result;
display x.L;
parameter totalAdTime;
totalAdTime = sum(I, x.L(I) );
display totalAdTime;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),1.970912e+06,3,21,NLP,KNITRO,0.078
1,Normal (1),OptimalLocal (2),1.970912e+06,3,21,NLP,CONOPT,0.016
2,Normal (1),OptimalLocal (2),1.970912e+06,3,21,NLP,IPOPT,0


In [4]:
# Display the total time purchased and the 
# amount of each ad purchased using the code below:
%gams_pull x totalAdTime
totalAdTime = totalAdTime[0]
c = [(x[i][0],x[i][1]) for i in range(0,len(x))]
print("totalAdTime = ",totalAdTime)
display(c)

totalAdTime =  20139.567425648293


[('ad1', 1879.119372732446),
 ('ad2', 300.72246675135904),
 ('ad3', 434.25035514573847),
 ('ad4', 1191.3331303718098),
 ('ad5', 1555.6128652379841),
 ('ad6', 1175.932519124003),
 ('ad7', 1067.690606046722),
 ('ad8', 953.2716519426701),
 ('ad9', 333.7437972270547),
 ('ad10', 343.8519293866644),
 ('ad11', 529.4673136079931),
 ('ad12', 477.79775002878955),
 ('ad13', 2171.704237172643),
 ('ad14', 2267.5256604834326),
 ('ad15', 951.0018830599919),
 ('ad16', 615.4953968309144),
 ('ad17', 375.73408542117613),
 ('ad18', 914.8816975429852),
 ('ad19', 1398.108323366737),
 ('ad20', 1202.3223841671784)]

### Write a GAMS qcp model that will determine the number of minutes of each advertising spot to purchase.  
You will need to add some auxiliary variables.  
Ensure your code works with at least two of the solvers (cplexd, mosek and gurobi).

In [5]:
%%gams
positive variable y(I);
variable objqcp;

equation eqqcpobj;
eqqcpobj.. objqcp =e= sum(I, c(I)*y(I)*y(I) ); 

equation eqqcpk1;
eqqcpk1.. K1 =l= sum(I, alpha(I)*y(I) ); 

equation eqqcpk2;
eqqcpk2.. K2 =l= sum(I, beta(I)*y(I) );

model advertqcp /eqqcpobj,eqqcpk1,eqqcpk2/;

option qcp=cplexd;
solve advertqcp using QCP min objqcp;
result("cplexd","ObjVal") = objqcp.L;

option qcp=mosek;
solve advertqcp using QCP min objqcp;
result("mosek","ObjVal") = objqcp.L;

*option qcp=gurobi;
*solve advertqcp using QCP min objqcp;
*result("gurobi","ObjVal") = objqcp.L;

display result;
display y.L;
totalAdTime = sum(I,y.L(I)*y.L(I) );
display totalAdTime;
x.L(I) = y.L(I)*y.L(I);
display x.L;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1.970912e+06,3,21,QCP,CPLEXD,0.015
1,Normal (1),Optimal Global (1),1.970912e+06,3,21,QCP,MOSEK,0


In [6]:
# Display the total time purchased and the 
# amount of each ad purchased using the code below:
%gams_pull x totalAdTime
totalAdTime = totalAdTime[0]
c = [(x[i][0],x[i][1]) for i in range(0,len(x))]
print("totalAdTime = ",totalAdTime)
display(c)

totalAdTime =  20139.56570590702


[('ad1', 1879.1126751054874),
 ('ad2', 300.7269045910207),
 ('ad3', 434.2544740584903),
 ('ad4', 1191.3307116658127),
 ('ad5', 1555.6081863567183),
 ('ad6', 1175.9301062195957),
 ('ad7', 1067.6889174809924),
 ('ad8', 953.2742602577965),
 ('ad9', 333.74799372665836),
 ('ad10', 343.85617981786805),
 ('ad11', 529.470970963317),
 ('ad12', 477.8014069603594),
 ('ad13', 2171.695824864411),
 ('ad14', 2267.5165812111472),
 ('ad15', 951.0044278619987),
 ('ad16', 615.4991345596814),
 ('ad17', 375.73805191231355),
 ('ad18', 914.8843416792656),
 ('ad19', 1398.1046255591334),
 ('ad20', 1202.3199310549535)]